In [ ]:
from utils import * 

/home/amir/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

            
logging.basicConfig(level=logging.INFO)

# # Load articles
# preprocessor = WikiProcessor()
# sources, citation_data = preprocessor.find_source_citations()

config = ExperimentConfig(collate_sample_size=50000,)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.add_special_tokens({
    'additional_special_tokens': [config.cite_token, config.ref_token]
})


# results = tokenize_sources(sources, citation_data, tokenizer, cache_dir="cache",)

# This will now use caching directly 
results = tokenize_sources(cache_path='./cache/tokenized_1caf5def_eb27a5477eaa3d549aebc4886f3717d1.pt')


# Create model config
model_config = CitationConfig(
    base_model_name=config.model_name,
    vocab_size=len(tokenizer),
    cite_token_id=tokenizer.convert_tokens_to_ids(config.cite_token),
    ref_token_id=tokenizer.convert_tokens_to_ids(config.ref_token),
    temperature=config.temperature,
)

# Initialize model
model = CitationModel(model_config)

loaded_model = torch.load('./experiments/best_citation_model.pt', weights_only=True)

model.load_state_dict(loaded_model['model_state_dict'])


# Usage example:
# Replace the previous training code with:
trained_model = train_citation_model(
    model=model,
    results=results,  # Pass raw results instead of dataloader
    tokenizer=tokenizer,
    config=config,
    num_epochs=15,
    weight_decay=0.01,
    learning_rate=1.5e-4,
    batch_size=200,
    train_ratio = 0.5,
    save_path="./experiments/best_citation_model.pt",
    temperatures = [1, .5,.2, .1, 0.07, 0.06, 0.05],
)


/home/amir/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:root:Loading cached tokenized results from ./cache/tokenized_1caf5def_eb27a5477eaa3d549aebc4886f3717d1.pt
/tmp/ipykernel_2203108/1310817513.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the

Using device: cuda


/tmp/ipykernel_2203108/615699325.py:732: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



Epoch 1/15
temperature changed to 1
Collating training data with new random masks...


Training:   0%|                                                                                                                       | 0/126 [00:00<?, ?it/s]/home/amir/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 126/126 [07:12<00:00,  3.43s/it, loss=2.41]



Average training loss: 5.8528

Running validation...


Computing similarities: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:07<00:00,  3.98it/s]



Validation metrics:
  Loss: 9.8985
  Accuracy (top-1): 0.0224
  Mean Reciprocal Rank: 0.0271
  Number of citations: 120513
  Number of unique targets: 30269

Top-k accuracy:
  k=1: 0.0063
  k=5: 0.0332
  k=10: 0.0657
  k=50: 0.1975
  k=100: 0.2637
  k=1000: 0.4346

Saved new best model to ./experiments/best_citation_model.pt
Best validation metrics so far:
  Loss: 9.8985
  Accuracy (top-1): 0.0224
  MRR: 0.0271

Epoch 2/15
temperature changed to 0.5
Collating training data with new random masks...


Training:  20%|███████████████████▋                                                                               | 25/126 [01:30<06:05,  3.62s/it, loss=5.11]


OutOfMemoryError: CUDA out of memory. Tried to allocate 636.00 MiB. GPU 0 has a total capacity of 23.59 GiB of which 572.19 MiB is free. Including non-PyTorch memory, this process has 20.03 GiB memory in use. Process 2204102 has 2.97 GiB memory in use. Of the allocated memory 14.70 GiB is allocated by PyTorch, and 5.02 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

/tmp/ipykernel_1688960/73700011.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load('./experiments/best_citation_model.pt')


<All keys matched successfully>

In [153]:
sample = dataset[0]
source_art_id = sample['source_art_id']
original_source = sources[source_art_id-1]
source_text = tokenizer.decode(sample['source_ids'], )
cited_art_ids = sample['cited_art_ids']

useless_chars = np.sum([c==']' for c in source_text])*2/len(source_text)
print('useless = ', useless_chars)
print(f"Source original: {source_art_id}:\n{original_source[:1000]}\n\n")
print('#'*50)
print(f"Source tokens decoded:\n{source_text[:]}\n\n")
print(f"Source attention mask: {(sample['attention_mask']==0).sum()}, ")

for i, target_art_id in enumerate(sample['target_art_ids']):
    target_art_ref = preprocessor.id2ref[target_art_id.item()]
    target_original = sources[target_art_id-1]
    target_text = tokenizer.decode(sample['target_ids'][i], )
    print(f"Target: id={target_art_id}:\n{target_original[:200]}...\n\n")
    print(f"Target tokens:\n{target_text[:]}\n\n")
target_art_ids = sample['target_art_ids']


useless =  0.1
Source original: 1:
'''April''' (Apr.) is the fourth [[month]] of the [[year]] in the [[Julian calendar|Julian]] and [[Gregorian calendar]]s, and comes between [[March]] and [[May]]. It is one of four months to have 30 [[day]]s.
April always begins on the same day of the week as [[July]], and additionally, [[January]] in leap years. April always ends on the same day of the week as [[December]].
== The Month ==
April comes between [[March]] and [[May]], making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as [[June]], [[September]] and [[November]] are later in the year.
April begins on the same day of the week as [[July]] every year and on the same day of the week as [[January]] in [[leap year]]s. April ends on the same day of the week as [[December]] every year, as each other's last days are exactly 35 weeks (245 days) apart.
In [[common year]]s, April starts on the same day of the week as [[October]] of the p